<a href="https://www.kaggle.com/code/sarraverse/02-association-rules?scriptVersionId=284161070" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/job-skills/skill_migration_public.csv
/kaggle/input/job-skills/all_jobs_clustered_full.csv
/kaggle/input/job-skills/association_rules_categories.csv
/kaggle/input/job-skills/all_jobs_mapped.csv
/kaggle/input/job-skills/all_jobs_clustered_sample_dbscan.csv
/kaggle/input/job-skills/association_rules_combined.csv
/kaggle/input/job-skills/all_jobs_clean_full.csv
/kaggle/input/job-skills/skill_migration_clean.csv
/kaggle/input/job-skills/association_rules_skills.csv
/kaggle/input/job-skills/all_jobs.csv


# ASSOCIATION RULES MINING (3 MODELS)


In [2]:
import pandas as pd
import re
import ast
from collections import Counter
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, apriori, association_rules

# Load skills dictionary
skills_meta = pd.read_csv("/kaggle/input/job-skills/skill_migration_clean.csv")

skill_dict = (
    skills_meta[["skill_group_name", "skill_group_category"]]
    .drop_duplicates()
    .set_index("skill_group_name")["skill_group_category"]
    .to_dict()
)

print(f"Loaded {len(skill_dict)} skills.")

# Load cleaned jobs data
path = "/kaggle/input/job-skills/all_jobs_mapped.csv"
df = pd.read_csv(path, nrows=200000)
print(f"Loaded dataset shape: {df.shape}")

# Ensure skill_list is properly formatted as list
def to_skill_list(x):
    if isinstance(x, list):
        return x
    if pd.isna(x):
        return []
    try:
        val = ast.literal_eval(x)
        if isinstance(val, list):
            return [str(s).strip().lower() for s in val]
    except:
        pass
    return [s.strip().lower() for s in str(x).split(",") if s.strip()]

df["skill_list"] = df["skill_list"].apply(to_skill_list)
df["skill_categories"] = df["skill_categories"].fillna("").astype(str)

# Define helper functions for parsing skills
def parse_skills(raw):
    if pd.isna(raw):
        return []
    text = str(raw).strip().lower()
    if not text:
        return []
    parts = re.split(r",|/|;|\||\+", text)
    return [p.strip() for p in parts if p.strip()]

def map_skill_categories(skill_list):
    cats = set()
    for s in skill_list:
        cat = skill_dict.get(s)
        if cat is None:
            cat = "other"
        cats.add(cat)
    if not cats:
        return ""
    return ",".join(sorted(cats))

def cat_list(cats_str):
    if not cats_str:
        return []
    return [c.strip().lower() for c in cats_str.split(",") if c.strip()]

# Prepare transactions for all models
print("\nPreparing transaction data...")

# Model A1: skills only
transactions_skills = df["skill_list"].tolist()
print(f"Model A1: {len(transactions_skills)} transactions with skills")

# Model A2: categories only
df["cat_list"] = df["skill_categories"].apply(cat_list)
transactions_cats = df["cat_list"].tolist()
print(f"Model A2: {len(transactions_cats)} transactions with categories")

# Model A3: skills + categories
def combined_list(row):
    return row["skill_list"] + row["cat_list"]

df["combined_list"] = df.apply(combined_list, axis=1)
transactions_combined = df["combined_list"].tolist()
print(f"Model A3: {len(transactions_combined)} transactions with skills + categories")

# Model A1: FP-Growth with skill-level rules
print("\n=== Running Model A1: Skill-Level Association Rules ===")
n_transactions = len(transactions_skills)
min_support = 0.01
min_confidence = 0.4
min_occurrences = 10

min_occ_from_support = max(1, int(min_support * n_transactions))
min_keep = max(min_occurrences, min_occ_from_support)

# 1) Count skill frequencies and filter rare skills
skill_counts = Counter(skill for tx in transactions_skills for skill in tx)
valid_skills = {skill for skill, cnt in skill_counts.items() if cnt >= min_keep}
filtered_transactions = [[s for s in tx if s in valid_skills] for tx in transactions_skills]

print(f"Kept skills: {len(valid_skills):,} (≥ {min_keep} occurrences)")

# 2) Encode transactions as a sparse matrix
te = TransactionEncoder()
te_ary = te.fit(filtered_transactions).transform(filtered_transactions, sparse=True)
skills_df = pd.DataFrame.sparse.from_spmatrix(te_ary, columns=te.columns_)

# 3) Run FP-Growth
freq_itemsets_A1 = fpgrowth(skills_df, min_support=min_support, use_colnames=True)

# 4) Derive association rules
rules_A1 = association_rules(freq_itemsets_A1, metric="confidence", min_threshold=min_confidence)

print(f"✅ Model A1: {len(rules_A1):,} rules found")

# Model A2: Category-level rules
print("\n=== Running Model A2: Category-Level Association Rules ===")
te = TransactionEncoder()
te_ary = te.fit(transactions_cats).transform(transactions_cats)
cats_df = pd.DataFrame(te_ary, columns=te.columns_)

freq_itemsets_A2 = apriori(cats_df, min_support=0.01, use_colnames=True)
rules_A2 = association_rules(freq_itemsets_A2, metric="confidence", min_threshold=0.4)

print(f"✅ Model A2: {len(rules_A2):,} rules found")

# Model A3: Combined rules
print("\n=== Running Model A3: Combined Association Rules ===")
n_transactions = len(transactions_combined)
min_support = 0.01
min_confidence = 0.4
min_occurrences = 10

min_occ_from_support = max(1, int(min_support * n_transactions))
min_keep = max(min_occurrences, min_occ_from_support)

# 1) Filter rare categories
cat_counts = Counter(cat for tx in transactions_combined for cat in tx)
valid_cats = {cat for cat, cnt in cat_counts.items() if cnt >= min_keep}
filtered_transactions = [[c for c in tx if c in valid_cats] for tx in transactions_combined]

print(f"Kept categories: {len(valid_cats):,} (≥ {min_keep} occurrences)")

# 2) Encode as sparse matrix
te = TransactionEncoder()
te_ary = te.fit(filtered_transactions).transform(filtered_transactions, sparse=True)
cats_df = pd.DataFrame.sparse.from_spmatrix(te_ary, columns=te.columns_)

# 3) Use FP-Growth
freq_itemsets_A3 = fpgrowth(cats_df, min_support=min_support, use_colnames=True)

# 4) Generate rules
rules_A3 = association_rules(freq_itemsets_A3, metric="confidence", min_threshold=min_confidence)

print(f"✅ Model A3: {len(rules_A3):,} rules found")

# Summary function
def summarize_rules(rules, name):
    print(f"\n=== {name} ===")
    print(f"Number of rules: {len(rules):,}")
    print(f"Support range: {rules['support'].min():.4f} → {rules['support'].max():.4f}")
    print(f"Confidence range: {rules['confidence'].min():.4f} → {rules['confidence'].max():.4f}")
    print(f"Lift range: {rules['lift'].min():.4f} → {rules['lift'].max():.4f}")
    
    # Show top 5 rules by confidence
    if len(rules) > 0:
        print("\nTop 5 rules by confidence:")
        top_rules = rules.sort_values('confidence', ascending=False).head(5)
        for idx, row in top_rules.iterrows():
            print(f"  {set(row['antecedents'])} → {set(row['consequents'])}")
            print(f"    Support: {row['support']:.4f}, Confidence: {row['confidence']:.4f}, Lift: {row['lift']:.4f}")

# Summarize all models
summarize_rules(rules_A1, "A1: skill-level")
summarize_rules(rules_A2, "A2: category-level")
summarize_rules(rules_A3, "A3: combined")

# Save results
print("\n💾 Saving association rules results...")
rules_A1.to_csv("/kaggle/working/association_rules_skills.csv", index=False)
rules_A2.to_csv("/kaggle/working/association_rules_categories.csv", index=False)
rules_A3.to_csv("/kaggle/working/association_rules_combined.csv", index=False)

print("✅ All association rules saved to files!")

Loaded 249 skills.
Loaded dataset shape: (200000, 9)

Preparing transaction data...
Model A1: 200000 transactions with skills
Model A2: 200000 transactions with categories
Model A3: 200000 transactions with skills + categories

=== Running Model A1: Skill-Level Association Rules ===
Kept skills: 171 (≥ 2000 occurrences)


/tmp/ipykernel_13/2373446715.py:108: FutureWarning: Allowing arbitrary scalar fill_value in SparseDtype is deprecated. In a future version, the fill_value must be a valid value for the SparseDtype.subtype.
  skills_df = pd.DataFrame.sparse.from_spmatrix(te_ary, columns=te.columns_)


✅ Model A1: 308 rules found

=== Running Model A2: Category-Level Association Rules ===
✅ Model A2: 22 rules found

=== Running Model A3: Combined Association Rules ===
Kept categories: 176 (≥ 2000 occurrences)


/tmp/ipykernel_13/2373446715.py:149: FutureWarning: Allowing arbitrary scalar fill_value in SparseDtype is deprecated. In a future version, the fill_value must be a valid value for the SparseDtype.subtype.
  cats_df = pd.DataFrame.sparse.from_spmatrix(te_ary, columns=te.columns_)


✅ Model A3: 7,147 rules found

=== A1: skill-level ===
Number of rules: 308
Support range: 0.0101 → 0.1013
Confidence range: 0.4010 → 0.9297
Lift range: 1.3813 → 41.6524

Top 5 rules by confidence:
  {'word'} → {'excel'}
    Support: 0.0112, Confidence: 0.9297, Lift: 27.6909
  {'customer service', 'leadership', 'problemsolving'} → {'communication'}
    Support: 0.0105, Confidence: 0.8965, Lift: 3.0875
  {'verbal communication'} → {'written communication'}
    Support: 0.0196, Confidence: 0.8697, Lift: 35.0674
  {'inclusion'} → {'diversity'}
    Support: 0.0143, Confidence: 0.8685, Lift: 41.6524
  {'problem solving', 'customer service', 'leadership'} → {'communication'}
    Support: 0.0118, Confidence: 0.8664, Lift: 2.9841

=== A2: category-level ===
Number of rules: 22
Support range: 0.0136 → 0.4745
Confidence range: 0.4756 → 1.0000
Lift range: 1.0017 → 1.2677

Top 5 rules by confidence:
  {'business skills'} → {'other'}
    Support: 0.1904, Confidence: 1.0000, Lift: 1.0023
  {'soft sk